# Проект классификация комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Магазину нужен инструмент, который будет искать токсичные комментарии, то есть классифицировать их как негативные и позитивные, и отправлять их на модерацию. 

**Задача**

Обучить модель классифицировать комментарии на позитивные и негативные. Метрика модели F1 должна быть не меньше 0.75

### Описание данных

В данных находятся:
- *text* в нём содержит текст комментария
- *toxic* — целевой признак, показывающий позитивный это или негативный комментарий

# 1. Подготовка

Для начала нужно подготовить комментарии для классифицирования.

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
import re
import nltk
import warnings

from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import notebook
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import f1_score, make_scorer



In [2]:
df_toxic = pd.read_csv('/datasets/toxic_comments.csv')
warnings.filterwarnings("ignore")

In [3]:
print(df_toxic.info())
print(df_toxic['toxic'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB
None
0    143346
1     16225
Name: toxic, dtype: int64


In [4]:
display(df_toxic.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Текст комментариев выглядит адекватным. Позитивных комментариев почти в 10 раз больше негативных

In [5]:
corpus = df_toxic['text'].values.astype('U')


In [8]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()

In [10]:
def clear_text(text):
    new_text = re.sub(r'[^a-z]',' ',text)
    split_text = new_text.split()
    return ' '.join(split_text)

Проведена очистка комментариев от небуквенных символов

In [11]:
lemm_data = []
for i in range(len(corpus)): 
    clean_text = clear_text(corpus[i])
    tokens = nltk.word_tokenize(clean_text)
    stemmer = EnglishStemmer(ignore_stopwords=False)
    lemm = ' '.join([stemmer.stem(token) for token in tokens])
    lemm_data.append(lemm)


In [17]:
print('Текст до лемматизации и очистки:', corpus[1])
print('Текст после лемматизации и очистки:', lemm_data[1])

Текст до лемматизации и очистки: d'aww! he matches this background colour i'm seemingly stuck with. thanks.  (talk) 21:51, january 11, 2016 (utc)
Текст после лемматизации и очистки: d aww he match this background colour i m seem stuck with thank talk januari utc


Проведена лемматизация текста.

In [18]:
target = df_toxic['toxic']

In [19]:
lemm_data_train, lemm_data_test, target_train, target_test = train_test_split(
    lemm_data, target, test_size = 0.2, random_state=12345)

In [20]:
print(target_train.value_counts())


0    114670
1     12986
Name: toxic, dtype: int64


In [21]:
count_tf_idf = TfidfVectorizer()
tf_idf_tr = count_tf_idf.fit_transform(lemm_data_train)

Векторизованы признаки с помощью TfidfVectorizer

In [22]:
tf_idf_te = count_tf_idf.transform(lemm_data_test)

In [23]:
features_train = tf_idf_tr
features_test = tf_idf_te

В ходе подготовки текста комментариев к обучению модели, была проведена его предобработка.

- Убраны лишние (небуквенные) символы, лемматизированы комментарии. 
- Данные разбиты на обучающую и тестовую выборки. 
- Признаки векторизованы.

# 2. Обучение

In [24]:
rows = []

In [25]:
grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
gs = GridSearchCV(LogisticRegression(class_weight='balanced'), param_grid=grid,
                  scoring='f1')

In [26]:
gs.fit(features_train,target_train)
print('param:', gs.best_params_, 'f1_score:', gs.best_score_)

param: {'C': 10.0, 'penalty': 'l2'} f1_score: 0.7695464434580939


In [27]:
lr = {'model':'LogisticRegression', 'f1_score': gs.best_score_}
rows.append(lr)

In [28]:
params = {'alpha':[1, 10, 200, 265, 270, 275, 500],
               'penalty':["l1","l2"]}

gs2 =  GridSearchCV(SGDClassifier(class_weight='balanced'), param_grid=params,
                    scoring='f1')

In [29]:
gs2.fit(features_train, target_train)
print('param:', gs2.best_params_, 'f1_score:', gs2.best_score_)

param: {'alpha': 1, 'penalty': 'l2'} f1_score: 0.12746218646236362


In [30]:
sg = {'model':'SGDClassifier', 'f1_score': gs2.best_score_}
rows.append(sg)

In [31]:
ridge_params = {'alpha':[1, 10, 200, 265, 270, 275, 500]}

gs3 =  GridSearchCV(RidgeClassifier(class_weight='balanced'), param_grid=ridge_params, scoring='f1')

In [32]:
gs3.fit(features_train, target_train)
print('param:', gs3.best_params_, 'f1_score:', gs3.best_score_)

param: {'alpha': 10} f1_score: 0.7062091191763115


In [33]:
rc = {'model':'RidgeClassifier', 'f1_score': gs3.best_score_}
rows.append(rc)

In [34]:
table_f1 = pd.DataFrame(rows)
print(table_f1.sort_values(by='f1_score', ascending=False).reset_index(drop=True))

                model  f1_score
0  LogisticRegression  0.769546
1     RidgeClassifier  0.706209
2       SGDClassifier  0.127462


Обучено 3 линейных модели. Лучший результат выдала Логистияеская Регрессия. Она показала удовлетворительный результат выше необходимого показателя - 0.75

# 3. Выводы

In [37]:
model = LogisticRegression(class_weight='balanced', C=10.0, penalty='l2')
model.fit(features_train, target_train)
predictions = model.predict(features_test)
f1_tfidf_te = f1_score(target_test, predictions)
print('значение f1 на тестовой выборке:', f1_tfidf_te)

значение f1 на тестовой выборке: 0.7686230248306998


**ИТОГ**

- Проведена предобработка данных. (Лемматизирован и векторизован текст)

- Обучено 3 линейных модели, чтобы отобрать из них ту, которая покажет лучший результат f1_score

- Модель линейная регрессия показала удовлетворительный результат на тестовой выборке, f1_score больше 0.75 (f1 = 0.769)

Модель сможет классифицировть комментарии на негативный и позитивный